# ws 03

In [ ]:
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

In [ ]:
X, y = make_blobs(n_samples=1000, centers=5, n_features=2, cluster_std=1.1, random_state=1)


In [ ]:
X_df = pd.DataFrame(X, columns=['weight', 'height'])
y_df = pd.DataFrame(y, columns=['class'])

df = pd.concat([X_df, y_df], axis=1)

target_map = {0:'horse', 1:'bird', 2:'fish', 3:'cat',4:'dog'}  # 
df['class'] = df['class'].map(target_map)

# df.head()
df.sample(5, random_state=100)

In [ ]:
sns.scatterplot(x='weight', y='height', data=df, s=70,
                hue=df['class'], style=df['class'])

plt.xticks([])
plt.yticks([])
plt.show()

In [ ]:
class_names, y = np.unique(df['class'], return_inverse=True)
class_names

In [ ]:
X = df.drop('class', axis=1)
X.head()

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_sc = sc.fit_transform(X)
X_sc[:4]

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_sc, y, test_size=0.25, random_state=1) #

X_train.shape, X_test.shape

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense 

model = Sequential()

model.add(Dense(5100, activation='relu', input_shape=(2,)))  

model.add(Dense(5, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', 
             metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=1, validation_split=0.25) # no1

In [ ]:
score = model.evaluate(X_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

In [ ]:
y_pred = model.predict(X_test)

y_pred[:4]
y_pred[:4].round(3)

In [ ]:
y_pred_cat = np.argmax(y_pred, axis=1)
y_pred_cat[:4]

In [ ]:
df_hist = pd.DataFrame.from_dict(history.history)

df_hist['loss'].plot(style='b--', label='Trainning')
df_hist['val_loss'].plot(style='r-', label='Validation')
plt.xlabel('Epoch')
plt.ylabel('Loss')


plt.legend()
plt.title('Loss')

plt.show()

In [ ]:
df_hist['accuracy'].plot(style='b--', label='Trainning')
df_hist['val_accuracy'].plot(label='Test')

plt.title('Model Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend()

plt.show()

In [ ]:
class Onehot2Int(object):
    def __init__(self, model):
        self.model = model

    def predict(self, X):
        y_pred = self.model.predict(X)
        return np.argmax(y_pred, axis=1)

model_no_ohe = Onehot2Int(model)    

In [ ]:
from mlxtend.plotting import plot_decision_regions

ax = plot_decision_regions(X_train, y_train, clf=model_no_ohe,
                          zoom_factor=1.)

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles, class_names, framealpha=0.5)

plt.title('Model Regions') 
plt.xticks([])
plt.yticks([])
plt.xlabel('weight')
plt.ylabel('height')
plt.show()

# ws 08

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='SGD', 
             metrics=['accuracy'])

In [ ]:
import tensorflow as tf
opt = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9)
model.compile(loss='sparse_categorical_crossentropy', 
              optimizer=opt, 
             metrics=['accuracy'])

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='RMSprop', 
             metrics=['accuracy'])

# ws 09

In [ ]:
X, y = make_blobs(n_samples=300, centers=5, n_features=2, cluster_std=1.5, random_state=1)


# ws 10

In [ ]:
from tensorflow.keras.regularizers import l1, l2, l1_l2

model = Sequential()

model.add(Dense(256, activation='relu', kernel_regularizer=l1(0.001), input_shape=(2,)))  
model.add(Dense(128, activation='relu', kernel_regularizer=l1(0.001)))  # 
model.add(Dense(64, activation='relu', kernel_regularizer=l1(0.001)))  # 

model.add(Dense(5, activation='softmax'))

# ws 11

In [ ]:
model = Sequential()

model.add(Dense(256, activation='relu', kernel_regularizer=l2(0.001), input_shape=(2,)))  
model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.001)))  
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.001)))  

model.add(Dense(5, activation='softmax'))

# ws 12

In [ ]:
model = Sequential()

model.add(Dense(256, activation='relu', kernel_regularizer=l2(0.01), input_shape=(2,)))  #.001
model.add(Dropout(.4))
model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.01)))  #
model.add(Dropout(.4))
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01)))  # 
model.add(Dropout(.4))

model.add(Dense(5, activation='softmax'))

# ws 13

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(monitor='val_loss', verbose=1, patience=20) # 

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

model = Sequential()

model.add(Dense(256, activation='relu', input_shape=(2,)))   # 
model.add(Dense(128, activation='relu'))  # 
model.add(Dense(64, activation='relu'))  # 

model.add(Dense(5, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', 
             metrics=['accuracy'])

history = model.fit(X_train, y_train, batch_size=256, validation_data=(X_test, y_test), 
                    epochs=1000, verbose=0, callbacks=[es])

# ws14

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint


es = EarlyStopping(monitor='val_loss', verbose=1, patience=20)
mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', verbose=1, save_best_only=True) # mode='auto'

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

model = Sequential()


model.add(Dense(256, activation='relu', input_shape=(2,))) 
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))  

model.add(Dense(5, activation='softmax'))
model.summary()